### Cálculo perigosidae Incêndio Florestal

In [ ]:
burn_areas = '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/aardidas/aa_1975_2021.shp'

burn_count = '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/aardidas/nburn.tif'

refrst = '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/reflmt/ref_raster.tif'

dem = '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/topografia/dem_pnse.tif'

lulc_maps = {
    '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/lulc/r_cos1995.tif' : 12,
    '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/lulc/r_cos2007.tif' : 3,
    '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/lulc/r_cos2010.tif' : 5,
    '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/lulc/r_cos2015.tif' : 3,
    '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/lulc/r_cos2018.tif' : 4,
}


perigosidade = '/home/jasp/mystuff/fireloc/firerisk_analysis/pnse_sample/risk_result.tif'

rules_file = '/home/jasp/fireloc-system/firerisk/perigosity_rules.json'

In [ ]:
import os

from glass.wenv.grs import run_grass
from glass.pys.oss  import fprop
from glass.rst.rcls.grs import rcls_rules
from glass.prop.rst      import count_cells
from firegis.cons.firerisk import ELEVATION_RULES, SLOPE_RULES, ASPECT_RULES

In [ ]:
# Create GRASS GIS Session

ws, loc = os.path.dirname(perigosidade), f"loc_{fprop(perigosidade, 'fn')}"

gb = run_grass(ws, grassBIN='grass78', location=loc, srs=refrst)

import grass.script.setup as gsetup

gsetup.init(gb, ws, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.it.shp import shp_to_grs
from glass.rst.surf.grs import slope, aspect
from glass.rst.rcls.grs import grs_rcls
from glass.dtt.torst import grsshp_to_grsrst
from glass.rst.alg import grsrstcalc
from glass.rst.zon.grs import rstatszonal

In [ ]:
# Import data into GRASS GIS

gdem = rst_to_grs(dem, fprop(dem, 'fn'), as_cmd=True)

lulcgrs = {rst_to_grs(
    lulc, fprop(lulc, 'fn'), as_cmd=True
) : lulc_maps[lulc] for lulc in lulc_maps}

gburn = shp_to_grs(burn_areas, fprop(burn_areas, 'fn'), asCMD=True)
rburn = grsshp_to_grsrst(gburn, 0, f"rst_{gburn}", cmd=True)

cburn = rst_to_grs(burn_count, fprop(burn_count, 'fn'), as_cmd=True)

In [ ]:
# Generate slope and aspect

rslope = slope(gdem, 'sloperst', data='degrees', api="grass")
raspec = aspect(gdem, 'aspectrst', from_north=True, api="grass")

In [ ]:
# Reclassify aspect, slope and elevation

rsts = {rslope : SLOPE_RULES, gdem : ELEVATION_RULES, raspec : ASPECT_RULES}

for r in rsts:
    # Create file with rules
    rf = rcls_rules(rsts[r], os.path.join(ws, loc, f"rules_{r}.txt"))
    
    # Reclassify
    rsts[r] = grs_rcls(r, rf, f"rcls_{r}", as_cmd=True)

In [ ]:
# Get number of cells
ncells = count_cells(refrst)

# Get number of burned cells
burn_rst = grs_to_rst(
    rburn, os.path.join(ws, loc, f"{rburn}.tif"),
    as_cmd=True, rtype=int
)
bcells = count_cells(burn_rst)

In [ ]:
print(ncells)
print(bcells)

In [ ]:
def get_lri(factor, lri):
    # Count number of burned cells by class
    burn_cells_cls = rstatszonal(factor, rburn, 'count', f"burn_{factor}")
    
    # Count number of cells by class
    cells_cls = rstatszonal(factor, factor, 'count', f"class_{factor}")
    
    # Create LRI raster for the given factor
    _lri = grsrstcalc((
        f"(double({burn_cells_cls}) / double({cells_cls}))"
        f" / "
        f"({str(int(bcells))}.0 / {str(int(ncells))}.0)"
    ), lri)
    
    return lri

In [ ]:
# Create RI Rasters

for r in rsts:
    rsts[r] = get_lri(rsts[r], f"lri_{rsts[r]}")

lulcri = {get_lri(k, f"lri_{k}") : lulcgrs[k] for k in lulcgrs}

In [ ]:
# Get final lri for lulc

_exp = []
years = 0
for k in lulcri:
    _exp.append(f"({k} * {str(lulcri[k])})")
    
    years += lulcri[k]
    
lulc_lri = grsrstcalc(f"({' + '.join(_exp)}) / {str(years)}", 'lri_lulc')

In [ ]:
# Sum all ri
rsts[lulc_lri] = lulc_lri

frst = grsrstcalc(' + '.join(list(rsts.values())), fprop(perigosidade, 'fn'))

In [ ]:
# Export result
grs_to_rst(frst, perigosidade, rtype=int)